# 1. First thing to do: Import the required libraries.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import yfinance as yf  #Yahoo Finance
import datetime
import pandas as pd
import matplotlib.pyplot as plt

# 2. Get an specific ticker

In [ ]:
#ASSET
ticker = 'AAPL'
my_ticker = yf.Ticker(ticker)

start_date = datetime.datetime(1990, 12, 31)
end_date = datetime.datetime(2024, 2, 29)

prices_hist = my_ticker.history(start=start_date,end=end_date)
prices_hist

# 3. Express the dividend as a nominal return

- The next step is to calculte the dividend as a percentage. That can be done easily dividing the dividends by the closing price. 
- That can be interpreted as an extra return that is paid in that moment. 

*For example*, you can see that Apple stock paid USD 0.24 on november 11th 2023. The closing price of the stock was 186.162445, which mean that the dividends represent a nominal return of 0.1289%.

In [ ]:
prices_hist['Perc_Dividends']=prices_hist['Dividends']/prices_hist['Close']
prices_hist.loc[prices_hist['Perc_Dividends']!=0]

# 4. Calculate the total return for each date

- To calculate the total return for each date we can simply add the dividend’s return plus the price return. 
- We will also assign a «zero» to the first row, since we don’t have a previous price to calculate a return.

In [ ]:
prices_hist['Return'] = prices_hist['Close']/ prices_hist['Close'].shift(1)-1+prices_hist['Perc_Dividends']
prices_hist['Return'].iloc[0] = 0
prices_hist

# 5. Calculate the cumulative total return

- Then we can use the function cumprod(). We can use it on the whole DataFrame applying the function on the quantity (1+ ‘Return’). 
- If you want to understand in greater detail how to perform these calculations, you can look at our course Introductory Data Science for Investing and Trading. 

In [ ]:
prices_hist['Cum_Return']=(1 + prices_hist['Return']).cumprod() - 1
prices_hist

# 6. Calculate a new quantity known as Wealth Index

This quantity is basically how an initial investment of USD100 has evolved throughout the years:

In [ ]:
prices_hist['Wealth_Index']=100*(1 + prices_hist['Cum_Return'])
prices_hist

#. 7. Calculate a new quantity known as Previous Peak

This quantity tracks the maximum from previous periods. If you get a new maximum, the previous maximum ceases to exists from that point in time and is replaced by each new maximum:

In [ ]:
prices_hist['Prev_Peak']=prices_hist['Wealth_Index'].cummax()
prices_hist

# 7. Calculate the Maximum Drawdown

- Maximum drawdown is basically the negative return you get from the actual wealth index when you compared it to the previous peak. 
- If you are at a new maximum this quantity will be zero. Otherwise you will get a negative number. 
- Maximum drawdown shows how much of your investment you would have lost if you have a long position in an asset and you didn’t liquidate it.

In [ ]:
prices_hist['Max_Drawdown']= (prices_hist['Wealth_Index']-prices_hist['Prev_Peak'])/prices_hist['Prev_Peak']
prices_hist

# 8. Chart of Maximum Drawdown

This is a very simple chart of the maximum drawdown. We show how to make nicer plots in other notebooks. For example, take a look here. 

In [ ]:
plt.plot(prices_hist['Max_Drawdown'])
plt.show()